In [1]:
import sys
sys.path.append('process_data/')

import holidays
us_holidays = holidays.UnitedStates()
import numpy as np
import pandas as pd
import pickle as pkl

from sklearn.model_selection import train_test_split

from util_data import generate_time_series
from util_data import get_reference
from setup import *

%load_ext autoreload
%autoreload 2

In [2]:
max_lookback = 6
predict_hzn = 1
difference = True

In [62]:
dataset = "rail_catchment"
if dataset == "census_tract":
    spatial_identifier = 'GEOID10'
    main_var = 'tnc'
if dataset == "rail_catchment":
    spatial_identifier = 'station_id'
    main_var = 'rail'

In [63]:
import os
files = [(name[-17:-11], name[-10:-4], name.split("_")[0], name) for name in os.listdir(project_data_dir+"data_processed/"+dataset+"/") if name[-4:]=='.csv']
files = pd.DataFrame(files, columns=['start_date','end_date','mode','name'])
files['start_date'] = pd.to_datetime(files['start_date'], format="%y%m%d")
files['end_date'] = pd.to_datetime(files['end_date'], format="%y%m%d")

In [64]:
def stitch_files(data_dir, files, start_date, end_date, date_col, preprocess=None):
    
    df = pd.DataFrame()
    for s,e,name in zip(files['start_date'],files['end_date'],files['name']):  
        if (end_date >= s) & (start_date <= e):
            df_chunk = pd.read_csv(data_dir+name)
            if preprocess is not None: 
                df_chunk = preprocess(df_chunk)
            if (end_date >= e) & (start_date <= s):
                print(name, 'all')
                df = pd.concat([df,df_chunk])
            else:
                print(name, 'part')
                df_chunk[date_col] = pd.to_datetime(df_chunk[date_col]).dt.date
                df = pd.concat([df,df_chunk[(df_chunk[date_col]>=start_date)&(df_chunk[date_col]<=end_date)]])

    return df


In [65]:
time_size = 1
ref_period = 7 * (96//time_size)

# start_date = pd.to_datetime('20190801').date()
# end_date = pd.to_datetime('20200301').date()

# start_date = pd.to_datetime('20200316').date()
# end_date = pd.to_datetime('20201017').date()

# start_date = pd.to_datetime('20200302')
# end_date = pd.to_datetime('20200315')

# start_date = pd.to_datetime('20200316')
# end_date = pd.to_datetime('20200329')

start_date = pd.to_datetime('20200622')
end_date = pd.to_datetime('20200705')

# start_date = pd.to_datetime('20200608')
# end_date = pd.to_datetime('20200621')

# start_date = pd.to_datetime('20201012')
# end_date = pd.to_datetime('20201025')


In [66]:
start_date = start_date - pd.Timedelta(7, "d")

data = {}
for mode in ['rail','bus','tnc','los']:
    data[mode] = stitch_files(project_data_dir+"data_processed/"+dataset+"/", files[files['mode']==mode], start_date, end_date, "day")
    print(mode, data[mode].columns.tolist())

rail_df_200316_201031.csv part


/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


rail ['ts', 'day', 'quarter_hour', 'station_id', 'count']
bus_rail_df_200316_201031.csv part


/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


bus ['station_id', 'ts', 'day', 'quarter_hour', 'bus_count']
tnc_rail_df_200316_201031.csv part


/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


tnc ['station_id', 'ts', 'day', 'quarter_hour', 'tnc_count']
los_df_200316_201031.csv part


/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


los ['station_id', 'day', 'quarter_hour', 'ts', 'num_schd_trp_15min', 'day_index']


In [67]:
start_date = start_date + pd.Timedelta(7, "d")

start_date

Timestamp('2020-06-22 00:00:00')

In [68]:
var_list = ['tnc','los','rail','bus']
var_list.remove(main_var)

df = data[main_var]
for mode in var_list:
#     df = pd.merge(df, data[mode], how='left', on=['ts','day','day_index','quarter_hour']+[spatial_identifier])
    df = pd.merge(df, data[mode], how='left', on=['ts','day','quarter_hour']+[spatial_identifier])
df.fillna(0, inplace=True)

# temporal aggregation
df['ts'] = df['ts'] // time_size
df = df.groupby(['ts','day','quarter_hour']+[spatial_identifier], as_index=False).sum()

if dataset == 'census_tract':
    with open(project_dir+"data/data_processed/select_tracts.pkl", "rb") as f:
        common_stations = pkl.load(f)
else:
    with open(project_dir+"data/data_processed/common_stations.pkl", "rb") as f:
        common_stations = pkl.load(f)
        
df = df[df[spatial_identifier].isin(common_stations)]

# libpysal will automatically order stations in ascending order when reading adj matrix
# sort to match
df.sort_values(by=['ts']+[spatial_identifier], inplace=True)

# form vectors for train test split

df_pivots = {
    'rail': pd.pivot_table(df, index='ts', columns=spatial_identifier, values='count', aggfunc=sum, fill_value=0),
    'bus': pd.pivot_table(df, index='ts', columns=spatial_identifier, values='bus_count', aggfunc=sum, fill_value=0),
    'tnc': pd.pivot_table(df, index='ts', columns=spatial_identifier, values='tnc_count', aggfunc=sum, fill_value=0),
    'los': pd.pivot_table(df, index='ts', columns=spatial_identifier, values='num_schd_trp_15min', aggfunc=sum, fill_value=0)}


In [69]:
timestamps = df_pivots[main_var].index.to_numpy()
ts_hours = (timestamps % (96//time_size)) / (4/time_size)
if dataset == 'rail_catchment':
    mask = (ts_hours >=6) & (ts_hours <= 22)
    order = ['rail','bus','tnc']
else:
    mask = (ts_hours < 3) | (ts_hours >= 8)    
    order = ['tnc','rail','bus']
    
# generate time series data w.r.t. lookback and prediction horizon
x, y, los, ts, ref = generate_time_series(data=[df_pivots[v] for v in order], 
                                   targets=[df_pivots[main_var]], 
                                   others=[df_pivots['los']], 
                                   ts=timestamps,
                                   offset=predict_hzn, 
                                   lookback=max_lookback,
                                   ref_ts=ref_period,
                                   difference=difference,
                                   remove_list=timestamps[~mask].tolist())
# there is only one target, others and reference
y = y[0]
los = los[0]
ref = ref[0]
# Remove holidays?

In [70]:
weather = stitch_files(project_data_dir+"data_processed/"+dataset+"/", files[files['mode']=='weather'], start_date, end_date, "DATE")


weather_200316_201031.csv part


/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


In [71]:
# # process weather information
ts_day = pd.DataFrame(ts // (96//time_size), columns=['DAY_INDEX'])

# weather = pd.read_csv(data_dir+"data_processed/weather_"+start_date.strftime("%y%m%d")+'_'+end_date.strftime("%y%m%d")+'.csv'.csv")
weather['TEMP_PCT_DIFF'] = np.abs((weather['TAVG'] - weather['TAVGAVG']) / weather['TAVGAVG'])
weather['PRCP'] = weather['PRCP'] / weather['PRCP'].max()

weather = pd.merge(ts_day, weather, on='DAY_INDEX', how='left') [['PRCP','TEMP_PCT_DIFF']].to_numpy()


In [72]:
if difference:
    differenced = "diff"
else:
    differenced = "raw"

with open(data_dir+"data_processed/"+dataset+"/"+start_date.strftime("%y%m%d")+'_'+end_date.strftime("%y%m%d")+"_"+
          str(predict_hzn)+"_"+str(time_size)+"_"+differenced+".pkl","wb") as f:
    pkl.dump(x, f)
    pkl.dump(ref,f)
    pkl.dump(los, f)
    pkl.dump(weather, f)
    pkl.dump(y, f)
    pkl.dump(ts, f)
    pkl.dump(common_stations, f)
    


In [ ]:
### Run this when time period is 20190801 - 20200301
### to get selected 59 TNC census tracts.
### Criterion: having more than 30 trips/h pre-COVID

all_tracts = tnc_pivot.columns.to_numpy()

reselect = all_tracts[tnc_pivot.mean()>7.5]

import geopandas as gpd
from pyproj import CRS
from shapely import wkt
from shapely.geometry import Polygon, Point

# shapefile of census blocks 2010 
gblk = pd.read_csv(data_dir+'data_raw/CensusBlockTIGER2010.csv')
# craete a geometric object
gblk['the_geom'] = gblk['the_geom'].apply(wkt.loads)
# create the geo dataframe
block_gdf = gpd.GeoDataFrame(gblk, geometry='the_geom')
# Calculate the area of the blocks/polygons
block_gdf.crs = CRS('epsg:4326')
block_gdf = block_gdf.to_crs("epsg:26916")

tract_gdf = block_gdf.dissolve(by=['STATEFP10','COUNTYFP10','TRACTCE10'], 
                               as_index=False)[['STATEFP10','COUNTYFP10','TRACTCE10','the_geom']]
tract_gdf['area'] = tract_gdf['the_geom'].map(lambda p:p.area)
tract_gdf['GEOID10'] = tract_gdf['STATEFP10'].astype(str)+"_"+tract_gdf['COUNTYFP10'].astype(str)+"_"+tract_gdf['TRACTCE10'].astype(str)

tract_gdf[tract_gdf['GEOID10'].isin(reselect)].plot()

with open(project_dir+"data/data_processed/select_tracts.pkl", "wb") as f:
    pkl.dump(list(reselect), f)